pacotes

In [73]:
import sqlite3 # biblioteca para trabalhar com banco de dados em sql (mais leve)
import pandas as pd
import csv

Criando conexão com o banco SQLite

In [74]:
con = sqlite3.connect('primeiro_banco') 
cur = con.cursor() # salva comandos na memória
print('Conectado ao banco!')

Conectado ao banco!


Criando tabela teste (caso não exista)

In [75]:
cur.execute("""
    CREATE TABLE IF NOT EXISTS Teste (
        id integer,
        name text
    )
"""
)

con.commit()

Inserindo valores

In [76]:
cur.execute(
    '''
    INSERT INTO Teste VALUES (
        1,
        'João'
    );
    '''
)

con.commit()

Consultar valores

In [77]:
query = '''
    SELECT * FROM Teste
'''

df = pd.read_sql_query(query, con) #consulta (query) e conexão com o banco (con)
df.head()

,id,name
0,1,João


Atualizar valores

In [78]:
cur.execute(
    '''
    UPDATE Teste SET name = 'Maria' WHERE id = 1;
    '''
)

#como eu criei 3 vezes a linha id = 1 e name = 'João', ele atualizou todas as linhas

con.commit 

df = pd.read_sql_query(query, con) 
df.head()

,id,name
0,1,Maria


Deletar valores

In [79]:
cur.execute(
    '''
    DELETE FROM Teste WHERE id = 1;
    '''
)

con.commit
#deletou tudo =) porque todos tinham a id = 1
df = pd.read_sql_query(query, con)
df.head()

,id,name


Criar uma nova tabela SQL a partir dos dados presente em um arquivo CSV

Primeiramente, criar a tabela no banco com o nome das colunas e o tipo de dado

In [80]:
cur.execute(
'''
    CREATE TABLE IF NOT EXISTS Customers (
        CustomerKey integer,
        FirstName text,
        LastName text,
        BirthDate date,
        MaritalStatus text,
        Gender text,
        EmailAddress text,
        AnnualIncome float,
        TotalChildren integer,
        EducationLevel text,
        Occupation text,
        HomeOwner text
    )
'''
)
con.commit()

Agora, alimentar a base com o arquivo CSV através de um looping:

In [81]:
with open('AdventureWorks_Customers.csv', 'r', encoding='latin-1') as cst:
    custormer = csv.DictReader(cst) #considera a primeira linha como cabeçalho e virgula como separador
    to_db = [
        (i['CustomerKey'], 
        i['FirstName'], 
        i['LastName'], 
        i['BirthDate'], 
        i['MaritalStatus'], 
        i['Gender'], 
        i['EmailAddress'], 
        i['AnnualIncome'], 
        i['TotalChildren'], 
        i['EducationLevel'], 
        i['Occupation'], 
        i['HomeOwner']) for i in custormer]

cur.executemany("INSERT INTO Customers (CUstomerKey, FirstName, LastName, BirthDate, MaritalStatus, Gender, EmailAddress, AnnualIncome, TotalChildren, EducationLevel, Occupation, HomeOwner) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);", to_db)

Consultar a tabela criada através do pandas

In [82]:
query = '''
    SELECT * FROM Customers WHERE HomeOwner = 'Y'
'''


df = pd.read_sql_query(query, con)
df.head()

,CustomerKey,FirstName,LastName,BirthDate,MaritalStatus,Gender,EmailAddress,AnnualIncome,TotalChildren,EducationLevel,Occupation,HomeOwner
0,11000,JON,YANG,4/8/1966,M,M,jon24@adventure-works.com,"$90,000",2,Bachelors,Professional,Y
1,11002,RUBEN,TORRES,8/12/1965,M,M,ruben35@adventure-works.com,"$60,000",3,Bachelors,Professional,Y
2,11004,ELIZABETH,JOHNSON,8/8/1968,S,F,elizabeth5@adventure-works.com,"$80,000",5,Bachelors,Professional,Y
3,11005,JULIO,RUIZ,8/5/1965,S,M,julio1@adventure-works.com,"$70,000",0,Bachelors,Professional,Y
4,11007,MARCO,MEHTA,5/9/1964,M,M,marco14@adventure-works.com,"$60,000",3,Bachelors,Professional,Y


Consulta através do SQL

In [83]:
set = con.execute('''
    SELECT * 
    FROM Customers 
    WHERE HomeOwner = 'Y' 
    LIMIT 5
''')

for i in set:
    print(i)

(11000, 'JON', 'YANG', '4/8/1966', 'M', 'M', 'jon24@adventure-works.com', '$90,000 ', 2, 'Bachelors', 'Professional', 'Y')
(11002, 'RUBEN', 'TORRES', '8/12/1965', 'M', 'M', 'ruben35@adventure-works.com', '$60,000 ', 3, 'Bachelors', 'Professional', 'Y')
(11004, 'ELIZABETH', 'JOHNSON', '8/8/1968', 'S', 'F', 'elizabeth5@adventure-works.com', '$80,000 ', 5, 'Bachelors', 'Professional', 'Y')
(11005, 'JULIO', 'RUIZ', '8/5/1965', 'S', 'M', 'julio1@adventure-works.com', '$70,000 ', 0, 'Bachelors', 'Professional', 'Y')
(11007, 'MARCO', 'MEHTA', '5/9/1964', 'M', 'M', 'marco14@adventure-works.com', '$60,000 ', 3, 'Bachelors', 'Professional', 'Y')


Fechar conexão

In [84]:
con.close()